<a href="https://colab.research.google.com/github/ShaunakSen/Deep-Learning/blob/master/Copy_of_2_1_ForwardAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Forward Mode Automatic Differentiation

Forward mode AD can simply be implemented by defining a class to represent [dual numbers](https://en.wikipedia.org/wiki/Dual_number) which hold the value and its derivative. The following skeleton defines a dual number and implements multiplication. 

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [3]:
import math

class DualNumber:
    def __init__(self, value, dvalue):
        self.value = value
        self.dvalue = dvalue

    def __str__(self):
        return str(self.value) + " + " + str(self.dvalue) + "ε"

    def __mul__(self, other):
        return DualNumber(self.value * other.value,
            self.dvalue * other.value + other.dvalue * self.value)
    
    def __add__(self, other):
        #TODO: finish me
        # YOUR CODE HERE
        
        # (a + be) + (x + ye) = (a + x) + e(b + y) 
        return DualNumber(self.value + other.value, self.dvalue + other.dvalue)
      
    def __sub__(self, other):
        #TODO: finish me
        # YOUR CODE HERE
        # (a + be) - (x + ye) = (a - x) + e(b - y) 
        return DualNumber(self.value - other.value, self.dvalue - other.dvalue)
      
    def __truediv__(self, other):
      
      # to compute (a+be)/(x+ye), we first compute conjugate of Dr ie (x - ye)
      
      Dr_conjugate = DualNumber(other.value, -other.dvalue)
      
      # next we multiply Nr and Dr with the conjugate
      
      denominator = other * Dr_conjugate
      
      # the denominator after multiplying is only left with the real part squared (x^2)
      
      denominator_value = denominator.value
      
      numerator = self*Dr_conjugate
      
      # print(numerator, denominator)
      
      result = DualNumber(numerator.value/denominator_value, numerator.dvalue/denominator_value)
      
      return result
    
    # TODO: add missing methods
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    
    
    def __pow__(self, other):
      
      # (a+eb)^(c+ed) = a^c + e.a^c[d.lna + (b*c)/a]: reference link: https://math.stackexchange.com/questions/1914591/dual-number-ab-varepsilon-raised-to-a-dual-power-e-g-ab-varepsilon
      
      a = self.value
      b = self.dvalue
      c = other.value
      d = other.dvalue
      
      real_part = a**c
      
      dual_part = a**c*(d*math.log(a) + (b*c)/a)
      
      return DualNumber(real_part, dual_part)
      
    
print(DualNumber(4,4)/DualNumber(5,3))

0.8 + 0.32ε


In [5]:
# Tests

print (DualNumber(1,0) + DualNumber(1,0) / DualNumber(1,0) - DualNumber(1,0)**DualNumber(1,0))


1.0 + 0.0ε


In [9]:
print (DualNumber(1,2)**DualNumber(3,2))

1 + 6.0ε


## Implementing math functions

We also need to implement some core math functions. Here's the sine function for a dual number:

In [15]:
def sin(x):
    return DualNumber(math.sin(x.value), math.cos(x.value)*x.dvalue)
print(DualNumber(4,3))
print(sin(DualNumber(4,3)))

4 + 3ε
-0.7568024953079282 + -1.960930862590836ε


__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [0]:
# TODO: implement additional math functions on dual numbers

def cos(x):
    # YOUR CODE HERE
    return DualNumber(math.cos(x.value), -math.sin(x.value)*x.dvalue)

def tan(x):
    # YOUR CODE HERE
    sec_sq = 1/(math.cos(x.value)**2)
    return DualNumber(math.tan(x.value), sec_sq*x.dvalue)

def exp(x):
    # YOUR CODE HERE
    return DualNumber(math.exp(x.value), math.exp(x.value)*x.dvalue)
    

In [0]:
# Tests
assert cos(DualNumber(0,0)).value == 1
assert tan(DualNumber(0,0)).value == 0
assert exp(DualNumber(0,0)).value == 1


## Time to try it out

We're now in a position to try our implementation.

__Task:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 

In [31]:
# YOUR CODE HERE
# raise NotImplementedError()

# we have to represent x and y in form of dual nos

# since we are calculating the derivative of z wrt x the real part of x will be 0.5 and dual part will be 1 

x = DualNumber(0.5, 1)

# since we are calculating the derivative of z wrt x the real part of y will be 4.2 and dual part will be 0


y = DualNumber(4.2, 0)

sin_x = sin(x)

print(x*y + sin_x)

2.579425538604203 + 5.077582561890373ε


__Task__: Differentiate the above function with respect to $x$ and write the symbolic derivatives in the following box. Verify the result computed above is correct by plugging-in the values into your symbolic gradient expression.

The above code gives us the result as: `2.579425538604203 + 5.077582561890373ε`

This means that the value of the function $z=x\cdot y+sin(x)$ is 2.579425538604203 and the value of the derivative of z wrt x is 5.077582561890373 at x = 0.5 and y = 4.2 

Let us verify if it is correct or not:

**Verification of the real part:**

The value of $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$:

$z = 0.5\cdot4.2 + sin(0.5) = 2.57942554$

Thus this result is verified

**Verification of the derivative part:**


$z=x\cdot y+sin(x)$

$\partial z/\partial x = y + cos(x)$

Substituting  x = 0.5 and y = 4.2:

$\partial z/\partial x = 0.877582562 + 4.2 = 5.07758256$

Thus, the result is verified


__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above) and store the derivative in the variable `dzdy` (just the derivative, not the Dual Number). Verify by hand that the result is correct.

In [32]:
# YOUR CODE HERE
x = DualNumber(0.5, 0)

y = DualNumber(4.2, 1)


dzdy = (x*y + sin(x)).dvalue

print('dz/dy:', dzdy)

dz/dy: 0.5


**Verification of the derivative part:**


$z=x\cdot y+sin(x)$

$\partial z/\partial y = x$

Substituting  x = 0.5 

$\partial z/\partial y =0.5$

Thus, the result is verified


In [0]:
#Tests
assert dzdy
assert type(dzdy) == float


__Task:__ Finally, use the code block below to experiment and test the other math functions and methods you created.

Some other math functions to be tested are:

- tan
- power
- exp

Let us find $\partial z/\partial x$ for each of the following functions at **x=2 and y=3** 
1. $z = x^{y}$

   $\partial z/\partial x = y.x^{y-1}$
   
   The result should be **8 + 12.0ε**

2. $z = tan(x) + e^{x}$

  $\partial z/\partial x = {sec}^{2}(x) + e^{x}$
  
  Computing the result with normal functions:
  
  

In [40]:
1/(math.cos(2)**2) + math.exp(2)

13.163455302972569

In [41]:
math.tan(2) + math.exp(2)

5.2040162356691315

The result of the 2nd function should be **5.2040162356691315 + 13.163455302972569ε**

Using Dual Numbers:

In [33]:
# YOUR CODE HERE

# test for 1st function

x = DualNumber(2, 1)

y = DualNumber(3, 0)

print(x**y)

# first function verified



# test for 2nd function:

print (tan(x) + exp(x))

# second function verified

8 + 12.0ε
5.2040162356691315 + 13.163455302972569ε
